In [ ]:
"""Intermolecular Interactions and Symmetry-Adapted Perturbation Theory"""

__authors__ = "Konrad Patkowski"
__email__   = ["patkowsk@auburn.edu"]

__copyright__ = "(c) 2008-2020, The Psi4Education Developers"
__license__   = "BSD-3-Clause"
__date__      = "2020-07-16"

Adapted by Jeffrey Schriber

Author: Konrad Patkowski, Auburn University (patkowsk@auburn.edu; ORCID: 0000-0002-4468-207X)

Copyright: Psi4Education Project, 2020"

# Intermolecular Interactions and Symmetry Adapted Perturbation Theory
# Weak intermolecular interactions 

In this activity, you will examine some properties of weak interactions between molecules. As the molecular subunits are not connected by any covalent (or ionic) bonds, we often use the term *noncovalent interactions*. Suppose we want to calculate the interaction energy between molecule A and molecule B for a certain geometry of the A-B complex (obviously, this interaction energy depends on how far apart the molecules are and how they are oriented). The simplest way of doing so is by subtraction (in the so-called *supermolecular approach*):

\begin{equation}
E_{\rm int}=E_{\rm A-B}-E_{\rm A}-E_{\rm B}
\end{equation}

where $E_{\rm X}$ is the total energy of system X, computed using our favorite electronic structure theory and basis set. A negative value of $E_{\rm int}$ means that A and B have a lower energy when they are together than when they are apart, so they do form a weakly bound complex that might be stable at least at very low temperatures. A positive value of $E_{\rm int}$ means that the A-B complex is unbound - it is energetically favorable for A and B to go their separate ways. 

Let's consider a simple example of two interacting helium atoms and calculate $E_{\rm int}$ at a few different interatomic distances $R$. You will use Psi4 to calculate the total energies that you need to perform subtraction. When you do so for a couple different $R$, you will be able to sketch the *potential energy curve* - the graph of $E_{\rm int}(R)$ as a function of $R$.

OK, but how should you pick the electronic structure method to calculate $E_{\rm A-B}$, $E_{\rm A}$, and $E_{\rm B}$? Let's start with the simplest choice and try out the Hartree-Fock (HF) method. In case HF is not accurate enough, we will also try the coupled-cluster method with single, double, and perturbative triple excitations - CCSD(T). If you haven't heard about CCSD(T) before, let's just state that it is **(1)** usually very accurate (it's even called the *gold standard* of electronic structure theory) and **(2)** very expensive for larger molecules. For the basis set, let's pick the augmented correlation consistent triple-zeta (aug-cc-pVTZ) basis of Dunning which should be quite OK for both HF and CCSD(T).


In [ ]:
# A simple Psi4 input script to compute the potential energy curve for two helium atoms

%matplotlib notebook
import time
import numpy as np
import scipy
from scipy.optimize import *
np.set_printoptions(precision=5, linewidth=200, threshold=2000, suppress=True)
import psi4
import matplotlib.pyplot as plt

# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
psi4.core.set_output_file('output.dat', False)

numpy_memory = 2

psi4.set_options({'basis': 'aug-cc-pVTZ',
              'e_convergence': 1e-10,
              'd_convergence': 1e-10,
              'INTS_TOLERANCE': 1e-15})



We need to collect some data points to graph the function $E_{\rm int}(R)$. Therefore, we set up a list of distances $R$ for which we will run the calculations (we go with 11 of them). For each distance, we need to remember three values ($E_{\rm A-B}$, $E_{\rm A}$, and $E_{\rm B}$). For this purpose, we will prepare two $11\times 3$ NumPy arrays to hold the HF and CCSD(T) results.

In [ ]:
distances = [4.0,4.5,5.0,5.3,5.6,6.0,6.5,7.0,8.0,9.0,10.0]
ehf = np.zeros((11,3))
eccsdt = np.zeros((11,3))


We are almost ready to crunch some numbers! One question though: how are we going to tell Psi4 whether we want $E_{\rm A-B}$, $E_{\rm A}$, or $E_{\rm B}$? 
We need to define three different geometries. The $E_{\rm A-B}$ one has two helium atoms $R$ atomic units from each other - we can place one atom at $(0,0,0)$ and the other at $(0,0,R)$. The other two geometries involve one actual helium atom, with a nucleus and two electrons, and one *ghost atom* in place of the other one. A ghost atom does not have a nucleus or electrons, but it does carry the same basis functions as an actual atom - we need to calculate all energies in the same basis set, with functions centered at both $(0,0,0)$ and $(0,0,R)$, to prevent the so-called *basis set superposition error*. In Psi4, the syntax `Gh(X)` denotes a ghost atom where basis functions for atom type X are located. 

Using ghost atoms, we can now easily define geometries for the $E_{\rm A}$ and $E_{\rm B}$ calculations.


In [ ]:
for i in range(len(distances)):
  dimer = psi4.geometry("""
  He 0.0 0.0 0.0
  --
  He 0.0 0.0 """+str(distances[i])+"""
  units bohr
  symmetry c1
  """)

  psi4.energy('ccsd(t)')   #HF will be calculated along the way
  ehf[i,0] = psi4.variable('HF TOTAL ENERGY')
  eccsdt[i,0] = psi4.variable('CCSD(T) TOTAL ENERGY')
  psi4.core.clean()

  monomerA = psi4.geometry("""
  He 0.0 0.0 0.0
  --
  Gh(He) 0.0 0.0 """+str(distances[i])+"""
  units bohr
  symmetry c1
  """)

  psi4.energy('ccsd(t)')   #HF will be calculated along the way
  ehf[i,1] = psi4.variable('HF TOTAL ENERGY')
  eccsdt[i,1] = psi4.variable('CCSD(T) TOTAL ENERGY')
  psi4.core.clean()

  monomerB = psi4.geometry("""
  Gh(He) 0.0 0.0 0.0
  --
  He 0.0 0.0 """+str(distances[i])+"""
  units bohr
  symmetry c1
  """)

  psi4.energy('ccsd(t)')   #HF will be calculated along the way
  ehf[i,2] = psi4.variable('HF TOTAL ENERGY')
  eccsdt[i,2] = psi4.variable('CCSD(T) TOTAL ENERGY')
  psi4.core.clean()


We have completed the $E_{\rm A-B}$, $E_{\rm A}$, or $E_{\rm B}$ calculations for all 11 distances $R$ (it didn't take that long, did it?). We will now perform the subtraction to form NumPy arrays with $E_{\rm int}(R)$ values for each method, converted from atomic units (hartrees) to kcal/mol, and graph the resulting potential energy curves using the matplotlib library. 

In [ ]:
#COMPLETE the two lines below to generate interaction energies. Convert them from atomic units to kcal/mol.
einthf = 
eintccsdt = 


print ('HF PEC',einthf)
print ('CCSD(T) PEC',eintccsdt)

plt.plot(distances,einthf,'r+',linestyle='-',label='HF')
plt.plot(distances,eintccsdt,'bo',linestyle='-',label='CCSD(T)')
plt.hlines(0.0,4.0,10.0)
plt.legend(loc='upper right')
plt.show()


*Questions* 
1. Which curve makes more physical sense?
2. Why does helium form a liquid at very low temperatures?
3. You learned in freshman chemistry that two helium atoms do not form a molecule because there are two electrons on a bonding orbital and two electrons on an antibonding orbital. How does this information relate to the behavior of HF (which does assume a molecular orbital for every electron) and CCSD(T) (which goes beyond the molecular orbital picture)?
4. When you increase the size of the interacting molecules, the CCSD(T) method quickly gets much more expensive and your calculation might take weeks instead of seconds. It gets especially expensive for the calculation of $E_{\rm A-B}$ because A-B has more electrons than either A or B. Your friend suggests to use CCSD(T) only for the easier terms $E_{\rm A}$ and $E_{\rm B}$ and subtract them from $E_{\rm A-B}$ calculated with a different, cheaper method such as HF. Why is this a really bad idea?

*To answer the questions above, please double click this Markdown cell to edit it. When you are done entering your answers, run this cell as if it was a code cell, and your Markdown source will be recompiled.*


A nice feature of the supermolecular approach is that it is very easy to use - you just need to run three standard energy calculations, and modern quantum chemistry codes such as Psi4 give you a lot of methods to choose from. However, the accuracy of subtraction hinges on error cancellation, and we have to be careful to ensure that the errors do cancel between $E_{\rm A-B}$ and $E_{\rm A}+E_{\rm B}$. Another drawback of the supermolecular approach is that it is not particularly rich in physical insight. All that we get is a single number $E_{\rm int}$ that tells us very little about the underlying physics of the interaction. Therefore, one may want to find an alternative approach where $E_{\rm int}$ is computed directly, without subtraction, and it is obtained as a sum of distinct, physically meaningful terms. Symmetry-adapted perturbation theory (SAPT) is such an alternative approach.

# Symmetry-Adapted Perturbation Theory (SAPT)

SAPT is a perturbation theory aimed specifically at calculating the interaction energy between two molecules. Contrary to the supermolecular approach, SAPT obtains the interaction energy directly - no subtraction of similar terms is needed. Moreover, the result is obtained as a sum of separate corrections accounting for the electrostatic, induction, dispersion, and exchange contributions to interaction energy, so the SAPT decomposition facilitates the understanding and physical interpretation of results.
- *Electrostatic energy* arises from the Coulomb interaction between charge densities of isolated molecules.
- *Induction energy* is the energetic effect of mutual polarization between the two molecules.
- *Dispersion energy* is a consequence of intermolecular electron correlation, usually explained in terms of correlated fluctuations of electron density on both molecules.
- *Exchange energy* is a short-range repulsive effect that is a consequence of the Pauli exclusion principle.

In this activity, we will explore the simplest level of the SAPT theory called SAPT0 (see [Parker:2014] for the definitions of different levels of SAPT). A particular SAPT correction $E^{(nk)}$ corresponds to effects that are of $n$th order in the intermolecular interaction and $k$th order in the intramolecular electron correlation. In SAPT0, intramolecular correlation is neglected, and intermolecular interaction is included through second order:

\begin{equation}
E_{\rm int}^{\rm SAPT0}=E^{(10)}_{\rm elst}+E^{(10)}_{\rm exch}+E^{(20)}_{\rm ind,resp}+E^{(20)}_{\rm exch-ind,resp}+E^{(20)}_{\rm disp}+E^{(20)}_{\rm exch-disp}+\delta E^{(2)}_{\rm HF}
\end{equation}

In this equation, the consecutive corrections account for the electrostatic, first-order exchange, induction, exchange induction, dispersion, and exchange dispersion effects, respectively. The additional subscript ''resp'' denotes that these corrections are computed including response effects - the HF orbitals of each molecule are relaxed in the electric field generated by the other molecule. The last term $\delta E^{(2)}_{\rm HF}$ approximates third- and higher-order induction and exchange induction effects and is taken from a supermolecular HF calculation.

Sticking to our example of two helium atoms, let's now calculate the SAPT0 interaction energy contributions using Psi4. In the results that follow, we will group $E^{(20)}_{\rm ind,resp}$, $E^{(20)}_{\rm exch-ind,resp}$, and $\delta E^{(2)}_{\rm HF}$ to define the total induction effect (including its exchange quenching), and group $E^{(20)}_{\rm disp}$ with $E^{(20)}_{\rm exch-disp}$ to define the total dispersion effect.

In [ ]:
distances = [4.0,4.5,5.0,5.3,5.6,6.0,6.5,7.0,8.0,9.0,10.0]
eelst = np.zeros((11))
eexch = np.zeros((11))
eind = np.zeros((11))
edisp = np.zeros((11))
esapt = np.zeros((11))

for i in range(len(distances)):
  dimer = psi4.geometry("""
  He 0.0 0.0 0.0
  --
  He 0.0 0.0 """+str(distances[i])+"""
  units bohr
  symmetry c1
  """)

  psi4.energy('sapt0')
  eelst[i] = psi4.variable('SAPT ELST ENERGY') * 627.509
  eexch[i] = psi4.variable('SAPT EXCH ENERGY') * 627.509
  eind[i] = psi4.variable('SAPT IND ENERGY') * 627.509
  edisp[i] = psi4.variable('SAPT DISP ENERGY') * 627.509
  esapt[i] = psi4.variable('SAPT TOTAL ENERGY') * 627.509
  psi4.core.clean()

plt.close()
plt.ylim(-0.2,0.4)
plt.plot(distances,eelst,'r+',linestyle='-',label='SAPT0 elst')
plt.plot(distances,eexch,'bo',linestyle='-',label='SAPT0 exch')
plt.plot(distances,eind,'g^',linestyle='-',label='SAPT0 ind')
plt.plot(distances,edisp,'mx',linestyle='-',label='SAPT0 disp')
plt.plot(distances,esapt,'k*',linestyle='-',label='SAPT0 total')
plt.hlines(0.0,4.0,10.0)
plt.legend(loc='upper right')
plt.show()

*Questions* 
1. What is the origin of attraction between two helium atoms?
2. For the interaction of two helium atoms, which SAPT terms are *long-range* (vanish with distance like some inverse power of $R$) and which are *short-range* (vanish exponentially with $R$ just like the overlap of molecular orbitals)?
3. The dispersion energy decays at large $R$ like $R^{-n}$. Find the value of $n$ by fitting a function to the five largest-$R$ results. You can use `scipy.optimize.curve_fit` to perform the fitting, but you have to define the appropriate function first.
Does the optimal exponent $n$ obtained by your fit agree with what you know about van der Waals dispersion forces? Is the graph of dispersion energy shaped like the $R^{-n}$ graph for large $R$? What about intermediate $R$?

*Do you know how to calculate $R^{-n}$ if you have an array with $R$ values? If not, look it up in the NumPy documentation!* 


In [ ]:
#COMPLETE the definition of function f below.
def f

ndisp = scipy.optimize.curve_fit(f,distances[-5:],edisp[-5:])
print ("Optimal dispersion exponent:",ndisp[0][0])


# Interaction between two water molecules

For the next part, you will perform the same analysis and obtain the supermolecular and SAPT0 data for the interaction of two water molecules. We now have many more degrees of freedom: in addition to the intermolecular distance $R$, we can change the relative orientation of two molecules, or even their internal geometries (O-H bond lengths and H-O-H angles). In this way, the potential energy curve becomes a multidimensional *potential energy surface*. It is hard to graph functions of more than two variables, so we will stick to the distance dependence of the interaction energies. Therefore, we will assume one particular orientation of two water molecules (a hydrogen-bonded one) and vary the intermolecular distance $R$ while keeping the orientation, and molecular geometries, constant. The geometry of the A-B complex has been defined for you, but you have to request all the necessary Psi4 calculations and extract the numbers that you need. To save time, we will downgrade the basis set to aug-cc-pVDZ and use MP2 (an approximate method that captures most of electron correlation) in place of CCSD(T).

*Hints:* To prepare the geometries for the individual water molecules A and B, copy and paste the A-B geometry, but use the Gh(O2)... syntax to define the appropriate ghost atoms. Remember to run `psi4.core.clean()` after each calculation.


In [ ]:
distances_h2o = [2.7,3.0,3.5,4.0,4.5,5.0,6.0,7.0,8.0,9.0]
ehf_h2o = np.zeros((10,3))
emp2_h2o = np.zeros((10,3))
psi4.set_options({'basis': 'aug-cc-pVDZ'})

for i in range(len(distances_h2o)):
  dimer = psi4.geometry("""
  O1
  H1 O1 0.96
  H2 O1 0.96 H1 104.5
  --
  O2 O1 """+str(distances_h2o[i])+""" H1 5.0 H2 0.0
  X O2 1.0 O1 120.0 H2 180.0
  H3 O2 0.96 X 52.25 O1 90.0
  H4 O2 0.96 X 52.25 O1 -90.0
  units angstrom
  symmetry c1
  """)

 
#COMPLETE the MP2 energy calculations for A-B, A, and B, and prepare the data for the graph.
#Copy and paste the A-B geometry, but use the Gh(O2)... syntax to define the appropriate ghost atoms for the A and B calculations. 
#Remember to run psi4.core.clean() after each calculation.

einthf_h2o  = 
eintmp2_h2o = 

print ('HF PEC',einthf_h2o)
print ('MP2 PEC',eintmp2_h2o)

plt.close()
plt.plot(distances_h2o,einthf_h2o,'r+',linestyle='-',label='HF')
plt.plot(distances_h2o,eintmp2_h2o,'bo',linestyle='-',label='MP2')
plt.hlines(0.0,2.5,9.0)
plt.legend(loc='upper right')
plt.show()

In [ ]:
eelst_h2o = np.zeros((10))
eexch_h2o = np.zeros((10))
eind_h2o = np.zeros((10))
edisp_h2o = np.zeros((10))
esapt_h2o = np.zeros((10))

#COMPLETE the SAPT calculations for 10 distances to prepare the data for the graph.

plt.close()
plt.ylim(-10.0,10.0)
plt.plot(distances_h2o,eelst_h2o,'r+',linestyle='-',label='SAPT0 elst')
plt.plot(distances_h2o,eexch_h2o,'bo',linestyle='-',label='SAPT0 exch')
plt.plot(distances_h2o,eind_h2o,'g^',linestyle='-',label='SAPT0 ind')
plt.plot(distances_h2o,edisp_h2o,'mx',linestyle='-',label='SAPT0 disp')
plt.plot(distances_h2o,esapt_h2o,'k*',linestyle='-',label='SAPT0 total')
plt.hlines(0.0,2.5,9.0)
plt.legend(loc='upper right')
plt.show()

We can now proceed with the analysis of the SAPT0 energy components for the complex of two water molecules. *Please edit this Markdown cell to write your answers.*
1. Which of the four SAPT terms are long-range, and which are short-range this time?
2. For the terms that are long-range and decay with $R$ like $R^{-n}$, estimate $n$ by fitting a proper function to the 5 data points with the largest $R$, just like you did for the two interacting helium atoms (using `scipy.optimize.curve_fit`). How would you explain the power $n$ that you obtained for the electrostatic energy?


In [ ]:
#COMPLETE the optimizations below. 
nelst_h2o = 
nind_h2o = 
ndisp_h2o = 
print ("Optimal electrostatics exponent:",nelst_h2o[0][0])
print ("Optimal induction exponent:",nind_h2o[0][0])
print ("Optimal dispersion exponent:",ndisp_h2o[0][0])

In the cell below, plot your three exponential functions over a reasonable range. I'll define the x-values for you, but you'll need to make the y-values with your fitted exponents.

In [ ]:
# the x-values
xvalues = np.linspace(1,8, 50)

# Now your plot:


plt.show()

We clearly have a favorable dipole-dipole interaction, which results in negative (attractive) electrostatic energy. This is how the origins of hydrogen bonding might have been explained to you in your freshman chemistry class: two polar molecules have nonzero dipole moments and the dipole-dipole interaction can be strongly attractive. However, your SAPT components show you that it's not a complete explanation: the two water molecules are bound not only by electrostatics, but by two other SAPT components as well. Can you quantify the relative (percentage) contributions of electrostatics, induction, and dispersion to the overall interaction energy at the van der Waals minimum? This minimum is the second point on your curve, so, for example, `esapt_h2o[1]` is the total SAPT interaction energy.


In [ ]:
#now let's examine the SAPT0 contributions at the van der Waals minimum, which is the 2nd point on the curve
#COMPLETE the calculation of percentages.
percent_elst = 
percent_ind  = 
percent_disp = 
print ('At the van der Waals minimum, electrostatics, induction, and dispersion')
print (' contribute %5.1f, %5.1f, and %5.1f percent of interaction energy, respectively.'
 % (percent_elst,percent_ind,percent_disp))


You have now completed some SAPT calculations and analyzed the meaning of different corrections. Can you complete the table below to indicate whether different SAPT corrections can be positive (repulsive), negative (attractive), or both, and why?


Type in your answers below.
COMPLETE this table. 

|SAPT term         |    Positive/Negative/Both?   |  Why?|
|-|-|-|
|Electrostatics|   |                              |      |
|Exchange|         |                              |      |
|Induction|        |                              |      |
|Dispersion|       |                              |      |

# Ternary diagrams

Higher levels of SAPT calculations can give very accurate interaction energies, but are more computationally expensive than SAPT0. SAPT0 is normally sufficient for qualitative accuracy and basic understanding of the interaction physics. One important use of SAPT0 is to *classify different intermolecular complexes according to the type of interaction*, and a nice way to display the results of this classification is provided by a *ternary diagram*.

The relative importance of attractive electrostatic, induction, and dispersion contributions to a SAPT interaction energy for a particular structure can be marked as a point inside a triangle, with the distance to each vertex of the triangle depicting the relative contribution of a given type (the more dominant a given contribution is, the closer the point lies to the corresponding vertex). If the electrostatic contribution is repulsive, we can display the relative magnitudes of electrostatic, induction, and dispersion terms in the same way, but we need the second triangle (the left one). The combination of two triangles forms the complete diagram and we can mark lots of different points corresponding to different complexes and geometries.

Let's now mark all your systems on a ternary diagram, in blue for two helium atoms and in red for two water molecules. What kinds of interaction are represented? Compare your diagram with the one pictured below, prepared for 2510 different geometries of the complex of two water molecules, with all kinds of intermolecular distances and orientations (this graph is taken from [Smith:2016]). What conclusions can you draw about the interaction of two water molecules at *any* orientation?


In [ ]:
def ternary(sapt, title='', labeled=True, view=True, saveas=None, relpath=False, graphicsformat=['pdf']):
#Adapted from the QCDB ternary diagram code by Lori Burns
    """Takes array of arrays *sapt* in form [elst, indc, disp] and builds formatted
    two-triangle ternary diagrams. Either fully-readable or dotsonly depending
    on *labeled*.
    """
    from matplotlib.path import Path
    import matplotlib.patches as patches

    # initialize plot
    plt.close()
    fig, ax = plt.subplots(figsize=(6, 3.6))
    plt.xlim([-0.75, 1.25])
    plt.ylim([-0.18, 1.02])
    plt.xticks([])
    plt.yticks([])
    ax.set_aspect('equal')

    if labeled:
        # form and color ternary triangles
        codes = [Path.MOVETO, Path.LINETO, Path.LINETO, Path.CLOSEPOLY]
        pathPos = Path([(0., 0.), (1., 0.), (0.5, 0.866), (0., 0.)], codes)
        pathNeg = Path([(0., 0.), (-0.5, 0.866), (0.5, 0.866), (0., 0.)], codes)
        ax.add_patch(patches.PathPatch(pathPos, facecolor='white', lw=2))
        ax.add_patch(patches.PathPatch(pathNeg, facecolor='#fff5ee', lw=2))

        # label corners
        ax.text(1.0,
                -0.15,
                u'Elst (−)',
                verticalalignment='bottom',
                horizontalalignment='center',
                family='Times New Roman',
                weight='bold',
                fontsize=18)
        ax.text(0.5,
                0.9,
                u'Ind (−)',
                verticalalignment='bottom',
                horizontalalignment='center',
                family='Times New Roman',
                weight='bold',
                fontsize=18)
        ax.text(0.0,
                -0.15,
                u'Disp (−)',
                verticalalignment='bottom',
                horizontalalignment='center',
                family='Times New Roman',
                weight='bold',
                fontsize=18)
        ax.text(-0.5,
                0.9,
                u'Elst (+)',
                verticalalignment='bottom',
                horizontalalignment='center',
                family='Times New Roman',
                weight='bold',
                fontsize=18)

    xvals = []
    yvals = []
    cvals = []
    geomindex = 0 # first 11 points are He-He, the next 10 are H2O-H2O
    for sys in sapt:
        [elst, indc, disp] = sys

        # calc ternary posn and color
        Ftop = abs(indc) / (abs(elst) + abs(indc) + abs(disp))
        Fright = abs(elst) / (abs(elst) + abs(indc) + abs(disp))
        xdot = 0.5 * Ftop + Fright
        ydot = 0.866 * Ftop
        if geomindex <= 10:
          cdot = 'b'
        else:
          cdot = 'r'
        if elst > 0.:
            xdot = 0.5 * (Ftop - Fright)
            ydot = 0.866 * (Ftop + Fright)
        #print elst, indc, disp, '', xdot, ydot, cdot

        xvals.append(xdot)
        yvals.append(ydot)
        cvals.append(cdot)
        geomindex += 1

    sc = ax.scatter(xvals, yvals, c=cvals, s=15, marker="o", 
                    edgecolor='none', vmin=0, vmax=1, zorder=10)

    # remove figure outline
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

    # save and show
    plt.show()
    return 1

sapt = []
for i in range(11):
  sapt.append([eelst[i],eind[i],edisp[i]])
for i in range(10):
  sapt.append([eelst_h2o[i],eind_h2o[i],edisp_h2o[i]])
idummy = ternary(sapt)
from IPython.display import Image
Image(filename='water2510.png')


Write your responses here: 



## Benzene Dimer Interactions

We'd expect the interactions in the benzene dimer to be qualitatively different in comparison to the water dimer. While there are many isomers of the benzene dimer, we'll look at one called the sandwich configuration, where the two benzene molecules meet face-to-face (rather than edge). 

In [ ]:
distances_bz = [2.7,3.0,3.5,4.0,4.5,5.0,6.0,7.0,8.0,9.0]
eelst_bz = np.zeros((10))
eexch_bz = np.zeros((10))
eind_bz  = np.zeros((10))
edisp_bz = np.zeros((10))
esapt_bz = np.zeros((10))
psi4.set_options({'basis': 'aug-cc-pVDZ'})

for i in range(len(distances_bz)):
    R = str(distances_bz[i])
    dimer = psi4.geometry("""
  0 1
C      0.000000    0.000000   -1.391500
C      1.205100    0.000000   -0.695700
C      1.205100    0.000000    0.695700
C      0.000000    0.000000    1.391500
C     -1.205100    0.000000    0.695700
C     -1.205100    0.000000   -0.695700
H      0.000000    0.000000   -2.471500
H      2.140400    0.000000   -1.235700
H      2.140400    0.000000    1.235700
H      0.000000    0.000000    2.471500
H     -2.140400    0.000000    1.235700
H     -2.140400    0.000000   -1.235700
--
0 1
C      0.000000   """+R+"""  -1.391500
C     -1.205100   """+R+"""   -0.695700
C     -1.205100   """+R+"""   0.695700
C      0.000000   """+R+"""    1.391500
C      1.205100   """+R+"""    0.695700
C      1.205100   """+R+"""   -0.695700
H      0.000000   """+R+"""   -2.471500
H     -2.140400   """+R+"""   -1.235700
H     -2.140400   """+R+"""    1.235700
H      0.000000   """+R+"""    2.471500
H      2.140400   """+R+"""    1.235700
H      2.140400   """+R+"""    -1.235700
  """)

    # Run the calculation here, store your energies and components
    
        

In [ ]:
plt.close()
#plt.ylim(-10.0,10.0)
plt.plot(distances_h2o,eelst_bz,'r+',linestyle='-',label='SAPT0 elst')
plt.plot(distances_h2o,eexch_bz,'bo',linestyle='-',label='SAPT0 exch')
plt.plot(distances_h2o,eind_bz,'g^',linestyle='-',label='SAPT0 ind')
plt.plot(distances_h2o,edisp_bz,'mx',linestyle='-',label='SAPT0 disp')
plt.plot(distances_h2o,esapt_bz,'k*',linestyle='-',label='SAPT0 total')
plt.hlines(0.0,2.5,9.0)
plt.legend(loc='upper right')
plt.show()

Questions:

1. What component is most dominant in these benzene dimer interactions? Is the same component dominant at all distances?

2. Explain what may be the origin of the dominant interaction component. Is it seen in the water or helium examples? Why or why not?

3. The benzene dimer can be a useful model for systems like benzene crystals. Describe another molecular system with a similar type of interaction. What interactions ar likely most important in determining the stability of an interaction in your proposed system. Is there anything different about your proposed system that may lead to increased or decreased binding energies?